In [1]:
from __future__ import unicode_literals

import sys
import os
from bs4 import BeautifulSoup
import urllib2
import cv2
from matplotlib import pyplot as plt
import matplotlib
from PIL import Image, ImageDraw, ImageFont

In [2]:
# from https://stackoverflow.com/questions/4460921/extract-the-first-paragraph-from-a-wikipedia-article-python
import re
import textwrap
import yaml
import urllib
import urllib2
import random

import wikipedia

In [3]:
# set utf-8
stdi, stdo, stde = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdin, sys.stdout, sys.stderr = stdi, stdo, stde
sys.getdefaultencoding()

'utf-8'


In [4]:
OUTPUT_PATH = '/home/will/sdb1/cvdata/wikipages500/'

In [5]:
DATA_SIZE = 500
IMG_SIZE = (227, 227)

In [6]:
LANG = {
    'en': 'https://en.wikipedia.org',
    'zh': 'https://zh.wikipedia.org'
}

In [7]:
class WikipediaError(Exception):
    pass

In [8]:
class Wikipedia:
    url_article = 'http://%s.wikipedia.org/w/index.php?action=raw&title=%s'
    url_image = 'http://%s.wikipedia.org/w/index.php?title=Special:FilePath&file=%s'
    url_search = 'http://%s.wikipedia.org/w/api.php?action=query&list=search&srsearch=%s&sroffset=%d&srlimit=%d&format=yaml'
   
    def __init__(self, lang):
        self.lang = lang
   
    def __fetch(self, url):
        request = urllib2.Request(url)
        request.add_header('User-Agent', 'Mozilla/5.0')
       
        try:
            result = urllib2.urlopen(request)
        except urllib2.HTTPError, e:
            raise WikipediaError(e.code)
        except urllib2.URLError, e:
            raise WikipediaError(e.reason)
       
        return result
   
    def article(self, article):
        url = self.url_article % (self.lang, urllib.quote_plus(article))
        content = self.__fetch(url).read()
       
        if content.upper().startswith('#REDIRECT'):
            match = re.match('(?i)#REDIRECT \[\[([^\[\]]+)\]\]', content)
           
            if not match == None:
                return self.article(match.group(1))
           
            raise WikipediaError('Can\'t found redirect article.')
       
        return content
   
    def image(self, image, thumb=None):
        url = self.url_image % (self.lang, image)
        result = self.__fetch(url)
        content = result.read()
       
        if thumb:
            url = result.geturl() + '/' + thumb + 'px-' + image
            url = url.replace('/commons/', '/commons/thumb/')
            url = url.replace('/' + self.lang + '/', '/' + self.lang + '/thumb/')
           
            return self.__fetch(url).read()
       
        return content
   
    def search(self, query, page=1, limit=10):
        offset = (page - 1) * limit
        url = self.url_search % (self.lang, urllib.quote_plus(query), offset, limit)
        content = self.__fetch(url).read()
       
        parsed = yaml.load(content)
        search = parsed['query']['search']
       
        results = []
       
        if search:
            for article in search:
                title = article['title'].strip()
               
                snippet = article['snippet']
                snippet = re.sub(r'(?m)<.*?>', '', snippet)
                snippet = re.sub(r'\s+', ' ', snippet)
                snippet = snippet.replace(' . ', '. ')
                snippet = snippet.replace(' , ', ', ')
                snippet = snippet.strip()
               
                wordcount = article['wordcount']
               
                results.append({
                    'title' : title,
                    'snippet' : snippet,
                    'wordcount' : wordcount
                })
       
        # yaml.dump(results, default_style='', default_flow_style=False,
        #     allow_unicode=True)
        return results

In [9]:
def unwiki(wiki):
    """
   Remove wiki markup from the text.
   """
    wiki = re.sub(r'(?i)\{\{IPA(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'(?i)\{\{Lang(\-[^\|\{\}]+)*?\|([^\|\{\}]+)(\|[^\{\}]+)*?\}\}', lambda m: m.group(2), wiki)
    wiki = re.sub(r'\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\{[^\{\}]+\}\}', '', wiki)
    wiki = re.sub(r'(?m)\{\|[^\{\}]*?\|\}', '', wiki)
    wiki = re.sub(r'(?i)\[\[Category:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[Image:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'(?i)\[\[File:[^\[\]]*?\]\]', '', wiki)
    wiki = re.sub(r'\[\[[^\[\]]*?\|([^\[\]]*?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r'\[\[([^\[\]]+?)\]\]', '', wiki)
    wiki = re.sub(r'(?i)File:[^\[\]]*?', '', wiki)
    wiki = re.sub(r'\[[^\[\]]*? ([^\[\]]*?)\]', lambda m: m.group(1), wiki)
    wiki = re.sub(r"''+", '', wiki)
    wiki = re.sub(r'(?m)^\*$', '', wiki)

    return wiki

def unhtml(html):
    """
   Remove HTML from the text.
   """
    html = re.sub(r'(?i)&nbsp;', ' ', html)
    html = re.sub(r'(?i)<br[ \\]*?>', '\n', html)
    html = re.sub(r'(?m)<!--.*?--\s*>', '', html)
    html = re.sub(r'(?i)<ref[^>]*>[^>]*<\/ ?ref>', '', html)
    html = re.sub(r'(?m)<.*?>', '', html)
    html = re.sub(r'(?i)&amp;', '&', html)

    return html

def punctuate(text):
    """
   Convert every text part into well-formed one-space
   separate paragraph.
   """
    text = re.sub(r'\r\n|\n|\r', '\n', text)
    text = re.sub(r'\n\n+', '\n\n', text)

    parts = text.split('\n\n')
    partsParsed = []

    for part in parts:
        part = part.strip()

        if len(part) == 0:
            continue

        partsParsed.append(part)

    return '\n\n'.join(partsParsed)


In [11]:
def article_urls_crawl(prefix, root_url, count):
    atc_list = set([])
    link_list = set([root_url])
    while True:
        url = random.choice(tuple(link_list))
        try:
            content = urllib.urlopen(url)
        except:
            continue
#         print urllib.urlopen(url).headers.getheader('Content-Type')
        soup = BeautifulSoup(content, "html.parser")
        base=soup.find('div', id="bodyContent")

        for link in BeautifulSoup(str(base), "html.parser").findAll("a"):
            if 'href' in link.attrs:  # Make sure it is link
                if link['href'].startswith('/wiki/') and prefix + link['href'] != url:
                    link_list.add(prefix + link['href'])
                if ( 
                    link['href'].startswith('/wiki/') and  # Make sure it is a wiki page
                    len(link['href'].split('/')) == 3 and  # Make sure it is an article
                    len(link['href'].split(':')) == 1      # Make sure it is not category
                ):
#                     print(link['href'])
                    title = str(link['href'])
                    title = urllib.unquote(title).decode('utf8')
                    print title
                    atc_list.add(prefix + title)
                    if len(atc_list) == count:
                        print('fetched ' + str(count) + ' article urls from ' + prefix + '\n')
                        return atc_list

urls_by_lang = {}

for key in LANG:
    urls_by_lang[key] = article_urls_crawl(LANG[key], LANG[key], DATA_SIZE)

/wiki/Wikipedia
/wiki/Free_content
/wiki/Encyclopedia
/wiki/English_language
/wiki/Shorwell_helmet
/wiki/Anglo-Saxons
/wiki/Shorwell
/wiki/Isle_of_Wight
/wiki/Anglo-Saxon_warfare
/wiki/Pattern-welded
/wiki/Hanging_bowl
/wiki/Benty_Grange_helmet
/wiki/Sutton_Hoo_helmet
/wiki/Coppergate_Helmet
/wiki/Pioneer_Helmet
/wiki/Staffordshire_helmet
/wiki/Franks
/wiki/Scandinavia
/wiki/Shorwell_helmet
/wiki/2017–18_Bergen_County_eruv_controversy
/wiki/Suillus_spraguei
/wiki/Megalodon
/wiki/Oriole_Park_at_Camden_Yards
/wiki/Hailey_Dawson
/wiki/Ceremonial_first_pitch
/wiki/Major_League_Baseball
/wiki/Robotics
/wiki/Biggs_jasper
/wiki/Silicon_dioxide
/wiki/Minna_Lammert
/wiki/Lilli_Lehmann
/wiki/Marie_Lehmann_(soprano)
/wiki/Rhinemaidens
/wiki/Das_Rheingold
/wiki/Bayreuth_Festival
/wiki/Richard_Wagner
/wiki/Arothron_reticularis
/wiki/Tetrodotoxin
/wiki/Saxitoxin
/wiki/Wang_Jian_(businessman)
/wiki/Hainan_Airlines
/wiki/Technological_University_Dublin
/wiki/Republic_of_Ireland
/wiki/Association_footb

/wiki/List_of_Byzantine_Emperors
/wiki/Histamenon
/wiki/List_of_Byzantine_emperors
/wiki/Byzantine_Empire
/wiki/Nikephoros_II_Phokas
/wiki/Basil_II
/wiki/Circa
/wiki/Constantinople
/wiki/Theodora,_daughter_of_Constantine_VII
/wiki/Dynasty
/wiki/Macedonian_Dynasty
/wiki/Greek_language
/wiki/Byzantine_Emperor
/wiki/Gunthertuch
/wiki/Kourkouas
/wiki/Armenians
/wiki/Matthew_of_Edessa
/wiki/Çemişgezek
/wiki/Fourth_Armenia
/wiki/Sophene
/wiki/Nikephoros_II_Phokas
/wiki/Phokadai
/wiki/Cappadocia
/wiki/Asia_Minor
/wiki/John_Kourkouas
/wiki/Melitene
/wiki/Bardas_Skleros
/wiki/Skleroi
/wiki/Leo_the_Deacon
/wiki/Madrid_Skylitzes
/wiki/Byzantine–Arab_wars_(780–1180)#Byzantine_resurgence.2C_963.E2.80.931025
/wiki/Abbasid_Caliphate
/wiki/Hamdanid
/wiki/Emirate_of_Aleppo
/wiki/Sayf_al-Dawla
/wiki/Aleppo
/wiki/Battle_of_Raban
/wiki/Romanos_II
/wiki/Theophano_(Byzantine_Empress)
/wiki/Michael_Bourtzes
/wiki/Assassination
/wiki/Svyatoslav_I
/wiki/Bardas_Sklerus
/wiki/Bardas_Phokas_the_Younger
/wiki/Theo

/wiki/York_(disambiguation)
/wiki/City_of_York_(disambiguation)
/wiki/City_status_in_the_United_Kingdom
/wiki/Unitary_authority
/wiki/Coat_of_arms_of_York
/wiki/Geographic_coordinate_system
/wiki/Countries_of_the_United_Kingdom
/wiki/Regions_of_England
/wiki/Yorkshire_and_the_Humber
/wiki/Ceremonial_counties_of_England
/wiki/North_Yorkshire
/wiki/Eboracum
/wiki/Circa
/wiki/Anno_Domini
/wiki/Unitary_authority
/wiki/City_status_in_the_United_Kingdom
/wiki/City_of_York_Council
/wiki/Local_government_in_England#Councillors_and_mayors
/wiki/Local_Government_Act_2000#Options_for_council_executive_forms
/wiki/Conservative_Party_(UK)
/wiki/Lord_Mayor_of_York
/wiki/Leader_of_the_City_of_York_Council
/wiki/Keith_Orrell_(British_politician)
/wiki/Liberal_Democrats_(UK)
/wiki/Conservative_Party_(UK)
/wiki/List_of_MPs_elected_in_the_United_Kingdom_general_election,_2015
/wiki/Rachael_Maskell
/wiki/Labour_Party_(UK)
/wiki/Julian_Sturdy
/wiki/Conservative_Party_(UK)
/wiki/List_of_English_districts_by

/wiki/Moesha
/wiki/Late_Antiquity
/wiki/Latin
/wiki/Greek_language
/wiki/Roman_province
/wiki/Balkans
/wiki/Danube_River
/wiki/Central_Serbia
/wiki/Kosovo
/wiki/Republic_of_Macedonia
/wiki/Northern_Bulgaria
/wiki/Northern_Dobruja
/wiki/Haemus_Mons
/wiki/Scardus
/wiki/Drinus
/wiki/Donaris
/wiki/Euxine
/wiki/Thracians
/wiki/Thracians
/wiki/Dacians
/wiki/Illyrians
/wiki/Moesi
/wiki/Roman_Empire
/wiki/Polity
/wiki/Burebista
/wiki/Getae
/wiki/Gaius_Scribonius_Curio
/wiki/Proconsul
/wiki/Macedonia_(Roman_province)
/wiki/Marcus_Licinius_Crassus_(consul_30_BC)
/wiki/Triumvirate
/wiki/Augustus
/wiki/Roman_province
/wiki/Aulus_Caecina_Severus_(suffect_consul_1_BC)
/wiki/Cassius_Dio
/wiki/Legatus
/wiki/Achaea_(Roman_province)
/wiki/Macedonia_(Roman_province)
/wiki/List_of_Dacian_kings
/wiki/Duras_(Dacian_king)
/wiki/Roman_emperor
/wiki/Domitian
/wiki/Tsibritsa
/wiki/Procurator_(Roman)
/wiki/Singidunum
/wiki/Belgrade
/wiki/Viminacium
/wiki/Kostolac
/wiki/Remesiana
/wiki/Bela_Palanka
/wiki/Vidin
/w

/wiki/Torun_(disambiguation)
/wiki/Leaning_Tower_of_Toruń
/wiki/Gothic_architecture
/wiki/Defensive_walls
/wiki/Vistula
/wiki/Philadelphia
/wiki/Tenement
/wiki/City_of_Angels_(disambiguation)
/wiki/Toruń_gingerbread
/wiki/Nicolaus_Copernicus
/wiki/Latin_language
/wiki/Poland
/wiki/Voivodeships_of_Poland
/wiki/Kuyavian-Pomeranian_Voivodeship
/wiki/Powiat
/wiki/Metropolitan_area
/wiki/Time_zone
/wiki/Central_European_Time
/wiki/UTC+1
/wiki/Daylight_saving_time
/wiki/Central_European_Summer_Time
/wiki/UTC+2
/wiki/Telephone_numbering_plan
/wiki/Polish_car_number_plates
/wiki/World_Heritage_Site
/wiki/Medieval_Town_of_Toruń
/wiki/List_of_World_Heritage_Sites_in_Europe
/wiki/German_language
/wiki/Poland
/wiki/Vistula
/wiki/Toruń_Voivodeship
/wiki/Pomeranian_Voivodeship
/wiki/Kuyavian-Pomeranian_Voivodeship
/wiki/Bydgoszcz
/wiki/Bydgoszcz–Toruń
/wiki/Teutonic_Knights
/wiki/Middle_Ages
/wiki/Hanseatic_League
/wiki/Brick_Gothic
/wiki/Mannerism
/wiki/Baroque_architecture
/wiki/Royal_city
/wiki/P

...

/wiki/中華人民共和國
/wiki/一級行政區
/wiki/中華民國
/wiki/福建省_(中華民國)
/wiki/福建_(消歧義)
/wiki/福州_(福建路)
/wiki/建州_(唐朝)
/wiki/中国一级行政区#中华人民共和国
/wiki/省_(行政區劃)
/wiki/省会
/wiki/福州市
/wiki/省委书记
/wiki/于伟国
/wiki/人大常委会
/wiki/于伟国
/wiki/省长
/wiki/唐登杰
/wiki/政协
/wiki/崔玉英
/wiki/面积
/wiki/中华人民共和国各省级行政区面积列表
/wiki/人口
/wiki/中华人民共和国各省级行政区人口列表
/wiki/人口密度
/wiki/总和生育率
/wiki/中华人民共和国各省级行政区总和生育率表
/wiki/方言
/wiki/国内生产总值
/wiki/人民币
/wiki/中华人民共和国省级行政区地区生产总值列表
/wiki/人均国内生产总值
/wiki/中国省级行政区人均地区生产总值列表
/wiki/中华人民共和国省级行政区地区生产总值列表
/wiki/人类发展指数
/wiki/中国省级行政区人类发展指数列表
/wiki/中國民族列表
/wiki/汉族
/wiki/畲族
/wiki/中华人民共和国境内地区邮政编码列表
/wiki/中华人民共和国电话区号
/wiki/市树
/wiki/市花
/wiki/中华人民共和国车辆牌照
/wiki/地级行政区
/wiki/县级行政区
/wiki/乡级行政区
/wiki/ISO_3166-2
/wiki/閩東語
/wiki/閩南語
/wiki/客家語
/wiki/中華人民共和國
/wiki/省级行政区
/wiki/浙江省
/wiki/江西省
/wiki/廣東省
/wiki/臺灣海峽
/wiki/臺灣
/wiki/副省级城市
/wiki/地级市
/wiki/平潭综合实验区
/wiki/福建地理
/wiki/海上丝绸之路
/wiki/郑和下西洋
/wiki/福建经济
/wiki/中华人民共和国省级行政区地区生产总值列表
/wiki/中華人民共和國
/wiki/中国一级行政区
/wiki/中国（福建）自由贸易试验区
/wiki/武夷山
/wiki/鼓浪屿
/wiki/清源山
/wiki/太姥山
/wiki/福建土楼
/wiki/安平桥
/wi

/wiki/创作共用
/wiki/江苏
/wiki/阳湖县
/wiki/苏联
/wiki/江苏
/wiki/阳湖县
/wiki/中国共产党
/wiki/江苏
/wiki/阳湖县
/wiki/常州
/wiki/武进
/wiki/上海商務印書館
/wiki/陈独秀
/wiki/邓中夏
/wiki/俞秀松
/wiki/沈雁冰
/wiki/沈雁冰
/wiki/中国共产党
/wiki/林伯渠
/wiki/张太雷
/wiki/张国焘
/wiki/杨贤江
/wiki/恽雨棠
/wiki/廖陈云
/wiki/张闻天
/wiki/孙冶方
/wiki/第一次国共合作
/wiki/五卅运动
/wiki/侯绍裘
/wiki/杨贤江
/wiki/上海商務印書館
/wiki/俞秀松
/wiki/莫斯科中山大学
/wiki/王明
/wiki/瞿秋白
/wiki/周恩来
/wiki/第三国际
/wiki/托派
/wiki/哈巴罗夫斯克
/wiki/康生
/wiki/真理报
/wiki/救国时报
/wiki/陈云
/wiki/商务印书馆
/wiki/地理坐标
/wiki/東亞
/wiki/中國大陸
/wiki/台灣
/wiki/南海
/wiki/東海
/wiki/太平洋
/wiki/中國大陸
/wiki/台灣
/wiki/中華民國
/wiki/中华人民共和国
/wiki/南海
/wiki/東海
/wiki/地壳
/wiki/福建
/wiki/台湾
/wiki/海峽
/wiki/东海
/wiki/南海
/wiki/亞洲
/wiki/太平洋
/wiki/东海
/wiki/西太平洋
/wiki/台湾
/wiki/新北市
/wiki/三貂角
/wiki/釣魚臺列嶼
/wiki/黃尾嶼
/wiki/福建省
/wiki/平潭岛
/wiki/屏東縣
/wiki/鵝鑾鼻
/wiki/福建省
/wiki/东山岛
/wiki/大陸棚
/wiki/大陆岛
/wiki/澎湖群島
/wiki/屏東縣
/wiki/小琉球
/wiki/更新世
/wiki/冰河期
/wiki/古人类
/wiki/北港
/wiki/菊石
/wiki/中生代
/wiki/標準化石
/wiki/中生代
/wiki/三叠纪
/wiki/侏罗纪
/wiki/白垩纪
/wiki/大理石
/wiki/燕山运动
/wiki/白垩纪
/wiki/古新世
/wiki

/wiki/俄罗斯行政区划
/wiki/经纬度
/wiki/地理坐标
/wiki/俄罗斯行政区划#下級行政區
/wiki/俄罗斯
/wiki/俄罗斯联邦主体
/wiki/哈巴罗夫斯克边疆区
/wiki/俄罗斯行政区划
/wiki/国家杜马
/wiki/俄罗斯2010年人口统计
/wiki/俄罗斯人口城市排名
/wiki/俄罗斯2010年人口统计
/wiki/俄罗斯人口城市排名
/wiki/人口密度
/wiki/維基共享資源
/wiki/满语
/wiki/满文转写方案
/wiki/俄语罗马化
/wiki/黑龍江
/wiki/烏蘇里江
/wiki/海参崴
/wiki/俄羅斯聯邦
/wiki/遠東聯邦管區
/wiki/行政中心
/wiki/哈巴罗夫斯克边疆区
/wiki/首府
/wiki/叶罗费·帕夫洛维奇·哈巴罗夫
/wiki/1989年蘇聯人口普查
/wiki/2002年俄羅斯人口普查
/wiki/唐朝
/wiki/722年
/wiki/唐玄宗
/wiki/靺鞨
/wiki/黑水都督府
/wiki/遼朝
/wiki/節度使
/wiki/明朝
/wiki/奴兒干都指揮使司
/wiki/后金
/wiki/清朝
/wiki/皇太極
/wiki/崇德
/wiki/漢語
/wiki/諧音_(語言學)
/wiki/滿洲源流考
/wiki/俄国
/wiki/沙皇
/wiki/亚历山大二世_(俄国)
/wiki/西伯利亞
/wiki/尼古拉·尼古拉耶维奇·穆拉维约夫-阿穆尔斯基
/wiki/黑龍江
/wiki/叶罗费·帕夫洛维奇·哈巴罗夫
/wiki/第二次鸦片战争
/wiki/北京条约#中俄北京條約
/wiki/清朝
/wiki/黑龍江
/wiki/烏蘇里江
/wiki/庫頁島
/wiki/牡丹江
/wiki/七台河市
/wiki/勃利县
/wiki/玉门关
/wiki/汉代
/wiki/唐代
/wiki/铁岭卫
/wiki/铁岭市
/wiki/中华人民共和国
/wiki/降水
/wiki/相对湿度
/wiki/日照時數
/wiki/無軌電車
/wiki/松花江
/wiki/绥芬河
/wiki/满洲里市
/wiki/黑龙江省
/wiki/沈阳市
/wiki/哈尔滨市
/wiki/阿穆尔河畔共青城
/wiki/雅库茨克
/wiki/馬加丹
/wiki/堪察加彼得巴甫洛夫斯克
/wik

/wiki/中华人民共和国
/wiki/鹰潭站
/wiki/厦门站
/wiki/轨距
/wiki/毫米
/wiki/標準軌
/wiki/閉塞_(鐵路)
/wiki/電氣化鐵路
/wiki/接触网供电
/wiki/南昌铁路局
/wiki/鹰潭站
/wiki/沪昆铁路
/wiki/资溪站
/wiki/光泽站
/wiki/邵武站
/wiki/顺昌站
/wiki/来舟站
/wiki/外洋站
/wiki/外福铁路
/wiki/青州站
/wiki/三明北站
/wiki/三元区站
/wiki/永安站
/wiki/麦园站
/wiki/漳龙坎铁路
/wiki/漳平站
/wiki/梅水坑站
/wiki/漳泉肖铁路
/wiki/华安站
/wiki/漳州北站
/wiki/漳州东站
/wiki/龙海站
/wiki/厦深铁路
/wiki/角美站
/wiki/东孚站
/wiki/前场站
/wiki/福厦铁路
/wiki/杏林站
/wiki/杏林大桥
/wiki/集杏海堤
/wiki/杏林大桥
/wiki/集美站
/wiki/杏林大桥
/wiki/高集海堤
/wiki/厦门高崎站
/wiki/厦门站
/wiki/铁路文化公园
/wiki/中国
/wiki/铁路
/wiki/江西
/wiki/鹰潭
/wiki/福建
/wiki/厦门
/wiki/沪昆铁路
/wiki/南平市
/wiki/福州
/wiki/来福铁路
/wiki/漳厦铁路
/wiki/高集海堤
/wiki/厦门站
/wiki/铁道部
/wiki/铁道兵
/wiki/漳州支线铁路
/wiki/桥梁
/wiki/隧道
/wiki/電氣化
/wiki/沙县
/wiki/万能站
/wiki/三明北站
/wiki/三明北站
/wiki/江西
/wiki/鹰潭
/wiki/资溪
/wiki/福建
/wiki/光泽
/wiki/邵武
/wiki/顺昌
/wiki/南平
/wiki/沙县
/wiki/三明
/wiki/永安市
/wiki/漳平
/wiki/华安
/wiki/长泰
/wiki/漳州
/wiki/龙海
/wiki/厦门
/wiki/沪昆铁路
/wiki/皖赣铁路
/wiki/峰福铁路
/wiki/漳龙坎铁路
/wiki/漳泉肖铁路
/wiki/厦深铁路
/wiki/龙厦铁路
/wiki/福厦铁路
/wiki/维基共享资源
/wiki/Ope

/wiki/福建省
/wiki/三明市
/wiki/梅列区
/wiki/地理坐标
/wiki/南昌铁路局
/wiki/永安车务段
/wiki/鹰厦铁路
/wiki/四等站_(中鐵)
/wiki/福建省
/wiki/三明市
/wiki/梅列区
/wiki/列西街道
/wiki/鹰潭站
/wiki/厦门站
/wiki/中国铁路
/wiki/上游站
/wiki/鹰潭站
/wiki/鹰厦铁路
/wiki/三元区站
/wiki/厦门站
/wiki/中华人民共和国铁道部
/wiki/鹰厦铁路
/wiki/鹰潭站
/wiki/鹰潭南站
/wiki/余家站
/wiki/上清站
/wiki/圳上站
/wiki/饶桥站
/wiki/资溪站
/wiki/铁关村站
/wiki/园岱站
/wiki/大禾山站
/wiki/华桥站
/wiki/大源村站
/wiki/光泽站
/wiki/和顺站
/wiki/莫口站
/wiki/药村站
/wiki/邵武站
/wiki/晒口站
/wiki/吴家塘站
/wiki/官墩站
/wiki/拿口站
/wiki/卫闽站
/wiki/富文站
/wiki/埔上站
/wiki/顺昌站
/wiki/洋口站
/wiki/建西站
/wiki/峡阳站
/wiki/照口站
/wiki/来舟站
/wiki/外洋站
/wiki/青州站
/wiki/万能站
/wiki/沙县站
/wiki/三明北站
/wiki/上游站
/wiki/三元区站
/wiki/荆西站
/wiki/上房山站
/wiki/贡川站
/wiki/西坑站
/wiki/益口站
/wiki/永安站
/wiki/永浆站
/wiki/桂口站
/wiki/西洋站
/wiki/打虎坑站
/wiki/岭头站
/wiki/柯周坑站
/wiki/城口站
/wiki/麦园站
/wiki/城门站_(新桥镇)
/wiki/钱坂站
/wiki/卓宅站
/wiki/漳平站
/wiki/芦芝站
/wiki/易坑站
/wiki/梅水坑站
/wiki/小集站
/wiki/西陂站
/wiki/华安站
/wiki/金山桥站
/wiki/溪南坂站
/wiki/利水站
/wiki/磹口站
/wiki/长泰站
/wiki/漳州东站
/wiki/龙海站
/wiki/角美站
/wiki/东孚站
/wiki/前场站
/wiki/杏林站
/wiki/集杏海堤
/wiki/

In [12]:
def getChinese(context):
    context = context.decode("utf-8") # convert context from str to unicode
    filtrate = re.compile(u'[^\u4E00-\u9FA5]') # non-Chinese unicode range
    context = filtrate.sub(r'', context) # remove all non-Chinese characters
    context = context.encode("utf-8") # convert unicode back to str
    return context

In [13]:
def generate_imgs(langs, urls_by_lang, count, fontsizerange):
    for key in langs:
        print 'generating imgs for ' + key
        i = 0
        while i < count:
            wikipedia.set_lang(key)
            res = None
            try:
                res = wikipedia.summary(random.choice(tuple(urls_by_lang[key])).split('/')[-1])
            except:
                continue
            
            text = res
        #     if key == 'en':
        #         for n in re.findall(ur'[a-zA-Z ]+',res):
        #             text = text + n
        #     elif key == 'zh':
        #         for n in re.findall(ur'[\u4e00-\u9fff]+',res):
        #             text = text + n
#             print text


            image = Image.new('L', IMG_SIZE, 255)
            draw = ImageDraw.Draw(image)
            if key == 'en':
                font = ImageFont.truetype('/usr/share/fonts/MyFonts/consola.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=23):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            elif key == 'zh':
                font = ImageFont.truetype('/usr/share/fonts/MyFonts/msyh.ttf', random.randint(fontsizerange[0], fontsizerange[1]))
                margin = 0
                offset = 0
                for line in textwrap.wrap(text, width=23):
                    draw.text((margin, offset), line, font=font, fill="#000000")
                    offset += font.getsize(line)[1]
            image.save(OUTPUT_PATH + key + '-' + str(i) + '-wiki.jpg')
            i += 1

In [14]:
generate_imgs(LANG, urls_by_lang, DATA_SIZE, (10, 50))

generating imgs for en


/home/will/.local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


generating imgs for zh


IOError: [Errno 2] No such file or directory: u'/home/will/sdb1/cvdata/wikipages/zh-3636-wiki.jpg'